In [2]:
from IPython import display
display.clear_output()

import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import roboflow

import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics 8.3.111  Python-3.11.3 torch-2.8.0.dev20250416+cu128 CUDA:0 (NVIDIA GeForce GTX 1050 Ti, 4096MiB)
Setup complete  (4 CPUs, 7.9 GB RAM, 214.7/222.9 GB disk)


In [ ]:
# Em apple sillicon, verificar se o MPS está disponível
import torch
print(torch.backends.mps.is_available())

In [ ]:
# Em GPU's da NVIDIA, verificar se existem núcleos CUDA disponíveis
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())

# Treino de um modelo

In [ ]:
# descarregar o dataset do roboflow, depois de etiquetadas as imagens e criado o dataset

roboflow.login()

# criar um ficheiro com a key da api do roboflow, ou simplesmente substituir abaixo
api_key =  os.getenv("ROBOFLOW_PRIVATE_API_KEY") # esta é a PrivateAPIKey do site que fui buscar às definições

rf = roboflow.Roboflow(api_key)

# substituir nome do workspace e do projeto (consegue-se ver isso pelo url do projeto no roboflow)
project = rf.workspace("public-vtwkd").project("pokemmo-tx0hm") # tem de ser tudo em minusculo aparentemente

# se versão do dataset > 1, substituir pela versão correspondente
dataset = project.version(2).download(model_format="yolov8", location="dataset") # location é o path onde vai guardar o dataset
# WARN: necessário verificar os paths no ficheiro data.yaml, após este ser descarregado

You are already logged into Roboflow. To make a different login,run roboflow.login(force=True).
loading Roboflow workspace...
loading Roboflow project...


In [ ]:
# treinar o modelo
# lista de modelos pre-treinados disponível em https://docs.ultralytics.com/models/yolov8/#performance-metrics
model = YOLO("yolov8s.pt")  # carregar o modelo pre-treinado que se descarregou

# Treinar o modelo
results = model.train(data='dataset/data.yaml', epochs=20, imgsz=640, device="cpu") # intel/windows

In [ ]:
# Avaliar o modelo no conjunto de validação
results = model.val(data="dataset/data.yaml", imgsz=640, device="cpu") # não testado

# Inferência

In [25]:
# selecionar a melhor versão do modelo fine-tuned
model = YOLO("runs/detect/train/weights/best.pt")

In [26]:
# prever em novas imagens
confidence_level = 0.1
input_path = 'captured_images'
output_path = 'detections'
class_names = model.names


for file in os.listdir(input_path):
    if file.lower().endswith((".png")) or file.lower().endswith((".jpg")) or file.lower().endswith((".jpeg")):
        image = cv2.imread(os.path.join(input_path, file))
        results = model.predict(source=image, conf=confidence_level)  # gerar previsões acima de determinada confiança, e guardar imagens

        
        output_filename = f"prediction_{file}"
        output_filepath = os.path.join(output_path, output_filename)

        for result in results:
            result.save(filename=output_filepath)
            print("==== Resultados Previsão ====")
            print("Imagem: "+os.path.join(input_path, file))
            boxes = result.boxes.xyxy.cpu().numpy()  # Bounding boxes (x_min, y_min, x_max, y_max)
            scores = result.boxes.conf.cpu().numpy()  # Score de confiança
            labels = result.boxes.cls.cpu().numpy()  # Índice da classe

            for i in range(len(boxes)):
                class_id = labels[i]
                class_label = class_names[class_id] if class_id in class_names else "Desconhecido"

                print(f"--- Objeto {i+1} ---")
                print(f"Class: {class_label} (ID: {class_id})")
                print(f"Coordenadas Bounding Box: {boxes[i]}")
                print(f"Confiança: {scores[i]:.4f}")
                print("-------------------")

            print("\n")


0: 352x640 4 oddishs, 1 spinarak, 1 water, 114.1ms
Speed: 40.6ms preprocess, 114.1ms inference, 39.6ms postprocess per image at shape (1, 3, 352, 640)
==== Resultados Previsão ====
Imagem: captured_images\Capturar3.PNG
--- Objeto 1 ---
Class: oddish (ID: 23.0)
Coordenadas Bounding Box: [     1319.6      278.15      1409.8       404.2]
Confiança: 0.5711
-------------------
--- Objeto 2 ---
Class: spinarak (ID: 32.0)
Coordenadas Bounding Box: [     202.49      112.47      278.22      200.09]
Confiança: 0.5489
-------------------
--- Objeto 3 ---
Class: oddish (ID: 23.0)
Coordenadas Bounding Box: [     703.82      141.24      817.75      296.42]
Confiança: 0.3265
-------------------
--- Objeto 4 ---
Class: oddish (ID: 23.0)
Coordenadas Bounding Box: [     1621.2      778.02      1706.6      926.21]
Confiança: 0.2984
-------------------
--- Objeto 5 ---
Class: oddish (ID: 23.0)
Coordenadas Bounding Box: [     618.31      180.87      695.88      306.56]
Confiança: 0.2638
------------------

# Inferência em tempo real

In [22]:
import os
import mss
import cv2
import numpy as np
import time

# xhost +  para permitir acesso ao display no linux
os.environ['DISPLAY'] = ':0' # para evitar erro de display, no linux
import pyautogui

In [ ]:
# Carregar o modelo
model = YOLO("runs/detect/train/weights/best.pt")


screen_width, screen_height = pyautogui.size()

# função que captura o ecrã e devolve a imagem
def capture_screen():
    with mss.mss() as sct:
        screenshot = sct.grab(sct.monitors[1])  # Capturar do monitor principal
        img = np.array(screenshot)  # Converter em imagem
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR) # Converter BGRA para RGB
        img_height, img_width, _ = img.shape

        # Opcionalmente, guardar a imagem capturada
        timestamp = time.strftime("%Y%m%d-%H%M%S-%f")  # Include microseconds
        img_path = os.path.join('captured_images/pyautogui', f"capture_{timestamp}.jpg")
        cv2.imwrite(img_path, img)

        return img, timestamp, img_width, img_height

while True:
    img, timestamp, img_width, img_height = capture_screen()
 
    #results = model.predict(source=img, save=True, save_txt=True, conf=0.1) 
    results = model(img)

    # Extract detections (bounding boxes)
    detections = results[0].boxes.xyxy  # Bounding boxes (x1, y1, x2, y2)

    if len(detections) > 0:
        print(f"Detetou {len(detections)} objetos.")

        # Opcionalmente, guardar imagem com as deteções
        annotated_frame = results[0].plot()  # Draw bounding boxes on the image
        result_path = os.path.join('detections', f"result_{timestamp}.jpg")
        cv2.imwrite(result_path, annotated_frame)

        for i, (x1, y1, x2, y2) in enumerate(detections.tolist()):
            # Calculate the center of the object
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)

            # converter coordenadas
            scaled_x = int((center_x / img_width) * screen_width)
            scaled_y = int((center_y / img_height) * screen_height)

            # mover o rato
            pyautogui.moveTo(scaled_x, scaled_y, duration=0.3)
            pyautogui.click()
            print(f"Moved to object {i+1} at ({scaled_x}, {scaled_y})")

            # Pausa breve, entre objetos
            time.sleep(0.25)  # Adjust delay as needed

    time.sleep(3)